In [2]:
import torchvision
from torchvision import transforms as T
from torchvision.io import read_image

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [4]:
!nvidia-smi

Fri Nov 25 14:32:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 526.98       Driver Version: 526.98       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   42C    P8     6W /  N/A |    410MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# load chainer model

In [3]:
from util import load_model, predict_trait, load_audio,load_video
data = './fbjDmltKVOM.000.mp4'
model = load_model()

# def pytorch model

In [ ]:
class Block_a(nn.Module):
    def __init__(self, ch_in, ch_out):
        super(Block_a, self).__init__()
        self.conv1 = 


class ResNet18(nn.Module):
    def __init__(self, n_class=1000, dropout_rate=0.0):
        super(ResNet18, self).__init__()
        
        self.n_class = n_class

        layers = [
            [16, 16, 3, 2, "RE", True, 16],
            [16, 24, 3, 2, "RE", False, 72],
            [24, 24, 3, 1, "RE", False, 88],
            [24, 40, 5, 2, "RE", True, 96],
            [40, 40, 5, 1, "RE", True, 240],
            [40, 40, 5, 1, "RE", True, 240],
            [40, 48, 5, 1, "HS", True, 120],
            [48, 48, 5, 1, "HS", True, 144],
            [48, 96, 5, 2, "HS", True, 288],
            [96, 96, 5, 1, "HS", True, 576],
            [96, 96, 5, 1, "HS", True, 576],
        ]

        self.init_conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(16),
            h_swish(inplace=True),
        )

        self.block = []
        for in_channels, out_channels, kernal_size, stride, nonlinear, se, exp_size in layers:
            self.block.append(MobileBlock(in_channels, out_channels, kernal_size, stride, nonlinear, se, exp_size))
        self.block = nn.Sequential(*self.block)


        self.out_conv1 = nn.Sequential(
            nn.Conv2d(96, 576, kernel_size=1, stride=1),
            SqueezeBlock(576),
            nn.BatchNorm2d(576),
            h_swish(inplace=True),

            nn.AdaptiveAvgPool2d(1),
        )
        self.out_conv2 = nn.Sequential(
            nn.Conv2d(576, 1024, kernel_size=1, stride=1),
            h_swish(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Conv2d(1024, self.n_class, kernel_size=1, stride=1),
        )


    def forward(self, x):
        out = self.init_conv(x)
        out = self.block(out)
        out = self.out_conv1(out)
        SRM = out
        out = self.out_conv2(out)
        out = out.view(out.shape[0], -1)
        return out,SRM